In [1]:
from RAG import run_maven, load_document_batch, get_rag_response

from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from dotenv import load_dotenv

load_dotenv()
import os
os.environ["OPENAI_API_KEY"] = "Ur key"



In [2]:
# load from directory
loader = DirectoryLoader("eval")
documents = loader.load()
print(documents)

This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSe

[Document(page_content='THESIS FOR THE DEGREE OF DOCTOR OF PHILOSOPHY\n\nFrom Industry to Practice: Can Users Tackle Domain Tasks with Augmented Reality?\n\nYUCHONG ZHANG\n\nDepartment of Computer Science and Engineering\n\nDivision of Interaction Design and Software Engineering\n\nCHALMERS UNIVERSITY OF TECHNOLOGY\n\nGöteborg, Sweden 2023\n\nFrom Industry to Practice: Can Users Tackle Domain Tasks with Augmented Reality?\n\nYUCHONG ZHANG Göteborg, Sweden 2023 ISBN 978-91-7905-784-8\n\nCOPYRIGHT © YUCHONG ZHANG, 2023\n\nDoktorsavhandlingar vid Chalmers tekniska högskola Ny serie Nr 5250 ISSN 0346-718X\n\nISSN 1652-0769 Technical Report 230D Division of Interaction Design and Software Engineering Department of Computer Science and Engineering Chalmers University of Technology SE-412 96 Göteborg, Sweden Telephone: +46 (0)31-772 1000\n\nPrinted by Chalmers Reproservice Göteborg, Sweden 2023\n\nFrom Industry to Practice: Can Users Tackle Domain Tasks with Augmented Reality? YUCHONG ZHANG D

In [3]:
text_splitter = CharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=150
)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 793, which is longer than the specified 450
Created a chunk of size 1208, which is longer than the specified 450
Created a chunk of size 859, which is longer than the specified 450
Created a chunk of size 526, which is longer than the specified 450
Created a chunk of size 676, which is longer than the specified 450
Created a chunk of size 486, which is longer than the specified 450
Created a chunk of size 1078, which is longer than the specified 450
Created a chunk of size 1007, which is longer than the specified 450
Created a chunk of size 2073, which is longer than the specified 450
Created a chunk of size 1431, which is longer than the specified 450
Created a chunk of size 535, which is longer than the specified 450
Created a chunk of size 757, which is longer than the specified 450
Created a chunk of size 585, which is longer than the specified 450
Created a chunk of size 683, which is longer than the specified 450
Created a chunk of size 771, which is longe

In [4]:
for chunk in chunks:
    chunk.metadata["file_name"] = chunk.metadata["source"]

In [5]:
'''from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.with_openai()

testset = generator.generate_with_langchain_docs(chunks, test_size=2, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})
'''
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.llms import Ollama

generator_llm = Ollama(model="mistral")
critic_llm = Ollama(model="mistral")
# critic_llm = ChatOpenAI(model="gpt-4")
embeddings = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-large")

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

testset = generator.generate_with_langchain_docs(documents, test_size=6, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})


embedding nodes:  32%|███▏      | 18/56 [00:04<00:07,  5.08it/s]

KeyboardInterrupt: 

In [18]:
from datasets import load_dataset

amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2")
amnesty_qa

/Users/allangamal/anaconda3/envs/ag/lib/python3.11/site-packages/datasets/load.py:1461: FutureWarning: The repository for explodinggradients/amnesty_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/explodinggradients/amnesty_qa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/Users/allangamal/anaconda3/envs/ag/lib/python3.11/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


DatasetDict({
    eval: Dataset({
        features: ['question', 'ground_truth', 'answer', 'contexts'],
        num_rows: 20
    })
})

In [19]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)


In [21]:
from ragas import evaluate

result = evaluate(
    amnesty_qa["eval"],
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result


Evaluating: 100%|██████████| 80/80 [02:35<00:00,  1.94s/it]


{'context_precision': 1.0000, 'faithfulness': 0.7745, 'answer_relevancy': 0.9677, 'context_recall': 1.0000}

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Chroma
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from prompt_template import prompt as prompty
import os.path
from subprocess import STDOUT,PIPE
from sys import stdin
import subprocess
from docingesterTemp import load_document_batch
import json
from langchain.docstore.document import Document

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
vector_database = Chroma.from_documents(
    chunks,
    embeddings
)
retriever = vectorstore.as_retriever()

NameError: name 'chunks' is not defined

In [ ]:
from prompt_template import prompt as prompty
prompt = prompty()

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | generator_llm
    | StrOutputParser()
)

In [ ]:
questions = testset.to_pandas()["question"].to_list()
ground_truth = testset.to_pandas()["ground_truth"].to_list()

KeyboardInterrupt: 

In [ ]:
from datasets import Dataset

questions = testset.to_pandas()["question"].to_list()
ground_truth = testset.to_pandas()["ground_truth"].to_list()

data = {"question": [], "answer": [], "contexts": [], "ground_truth": ground_truth}

for query in questions:
    data["question"].append(query)
    data["answer"].append(rag_chain.invoke(query))
    data["contexts"].append([doc.page_content for doc in retriever.get_relevant_documents(query)])

dataset = Dataset.from_dict(data)

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset,
    metrics=[
        context_relevancy,
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

In [ ]:
result.to_pandas()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

df = result.to_pandas()

heatmap_data = df[['context_relevancy', 'context_precision', 'context_recall', 'faithfulness', 'answer_relevancy']]

cmap = LinearSegmentedColormap.from_list('green_red', ['red', 'green'])

plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data, annot=True, fmt=".2f", linewidths=.5, cmap=cmap)

plt.yticks(ticks=range(len(df['question'])), labels=df['question'], rotation=0)

plt.show()